**Алгоритм**

Необходимо разделить весь путь каждой траектории на одинаковое количество отрезков. На стыках между каждыми двумя отрезками лежат точки. Суть алгоритма состоит в том, что дистанции между соответствующими точками из каждой траектории будут маленькие, если траектории близкие, и большие, если траектории отличаются сильно. Чем больше таких точек будет, тем более точный результат выдаст алгоритм. Этот самый результат представляет собой отношение суммы дистанций между точками к минимуму из квадратов длин траекторий. Его значение лежит в диапазоне [0; 1]. Чем ближе к нулю, тем траектории более похожи друг на друга. Чем ближе к единице - тем они различнее.

In [50]:
import pandas as pd
import numpy as np

In [51]:
# функция вычисления расстояния между точками
def euclidean_distance(p1, p2):
  return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [52]:
# вычисляем дистанцию между нынешней точкой траектории и предыдущей
dataset = pd.read_csv('traks.csv', sep =  ';')

def get_prev_point_dist(row):
  if row.name == 0 or row['track'] !=  dataset.loc[row.name-1]['track']:
    return 0
  prev = dataset.loc[row.name-1]
  return euclidean_distance((prev['x'], prev['y']), (row['x'],row['y']))

dataset['dist_to_prev_point'] = dataset.apply(get_prev_point_dist, axis = 1)
dataset.head()

,track,time,x,y,dist_to_prev_point
0,1,10:32:13,1598,526,0.000000
1,1,10:32:14,1524,544,76.157731
2,1,10:32:15,1441,557,84.011904
3,1,10:32:16,1357,392,185.151289
4,1,10:32:17,1395,573,184.945938


In [53]:
coords_of_traks = {num: {'len': 0, 'points':[]} for num in dataset['track'].unique()}
coords_of_traks

{1: {'len': 0, 'points': []},
 2: {'len': 0, 'points': []},
 3: {'len': 0, 'points': []},
 4: {'len': 0, 'points': []}}

In [54]:
# группируем точки по номеру траектории
for idx, row in dataset.iterrows():
  coords_of_traks[row['track']]['points'].append([row['x'], row['y'], row['dist_to_prev_point']])
  coords_of_traks[row['track']]['len'] += row['dist_to_prev_point']

In [55]:
# объект-обработчик траектории
class TrakHandler:
  def __init__(self, trak, trak_len, k):
    # k - количество точек, которые алгоритму предстоит найти

    self.iterator = 0 # нынешнее место нахождения объекта
    self.trak_len = trak_len # длина траектории
    self.trak = trak # набор точек
    self.step_size = self.trak_len/k # размер шага

    self.cure_len = 0 # текущая длина от начала траектории до последней точки,
    # найденной объектом
    self.cure_step = self.step_size # текущий шаг 
    self.result_point = None # последняя найденная точка
    self.is_find_point = False # флаг найденной точки
    self.is_done = False # флаг завершения прохода по траектории
  
  # функция условия увеличения шага
  def increase_step(self):
    if self.cure_step + self.step_size < self.trak_len:
      self.cure_step += self.step_size
    else:
      self.is_done = True

  # функция поиска следующей точки
  def find_next_result_point(self, i):
    if self.cure_len + self.trak[i][2] < self.cure_step:
      self.cure_len += self.trak[i][2]
      self.is_find_point = False
      return self.is_find_point 
    coef = (self.cure_step - self.cure_len) / (self.cure_len + self.trak[i][2] - self.cure_len) 
    self.result_point = (self.trak[i][0]*coef, self.trak[i][1]*coef) # вычисляем в каком месте будет лежать следующая точка
    self.is_find_point = True
    return self.is_find_point 

# вычисление коэффициента схожести траекторий
def calc_similarity_of_traks(k, trak1, trak2):
  # k - количество точек на каждой траектории, trak1, trak2 - последовательности точек и дистанций между ними
  t1 = TrakHandler(trak1['points'], trak1['len'], k)
  t2 = TrakHandler(trak2['points'], trak2['len'], k)

  sum_dist = 0. # сумма длин
  while True:
    while True:
      # если в пределах следующего отрезка нет искомой точки, то делаем большой шаг вперед по первой последовательности отрезков
      if not t1.find_next_result_point(t1.iterator) and t1.iterator < len(t1.trak):
        t1.iterator += 1
      # то же справедливо для второй последовательности
      elif not t2.find_next_result_point(t2.iterator) and t2.iterator < len(t2.trak):
        t2.iterator += 1
      # если нашли соответствующие точки - вычисляем дистанцию между ними
      elif t1.is_find_point and t2.is_find_point:
        sum_dist += euclidean_distance(t1.result_point, t2.result_point)
        break
    t1.increase_step()
    t2.increase_step()

    if t1.is_done and t2.is_done:
      coeff = min(t1.trak_len**2, t2.trak_len**2)
      res = sum_dist/coeff
      # в тех частных случаях, когда алгоритм выдает значение больше единицы, 
      # траектории либо сильно отличаются по длине, либо расстояние между
      # траекториями больше, чем длина одной из траекторий. В любом из 
      # этих случаев траектории считаются максимально отличными друг от друга
      # и результату сравнения присваивается значение 1.
      if res > 1: 
        res = 1
      return sum_dist/coeff

In [56]:
points = 10000
score_1_3 = calc_similarity_of_traks(points, coords_of_traks[1], coords_of_traks[3])
score_1_2 = calc_similarity_of_traks(points, coords_of_traks[1], coords_of_traks[2])
score_1_4 = calc_similarity_of_traks(points, coords_of_traks[1], coords_of_traks[4])
score_2_4 = calc_similarity_of_traks(points, coords_of_traks[2], coords_of_traks[4])
score_3_4 = calc_similarity_of_traks(points, coords_of_traks[3], coords_of_traks[4])

worst = max(score_1_3, score_1_2, score_1_4, score_2_4, score_3_4)
sum = score_1_3 + score_1_2 + score_1_4 + score_2_4 + score_3_4

print('Обычная оценка / Стандартизированная оценка')
print('1-3 score: ', score_1_3,'1-3 normalized_score', score_1_3/worst)
print('1-2 score: ', score_1_2,'1-2 normalized_score', score_1_2/worst)
print('1-4 score: ', score_1_4,'1-4 normalized_score', score_1_4/worst)
print('2-4 score: ', score_2_4,'2-4 normalized_score', score_2_4/worst)
print('3-4 score: ', score_3_4,'3-4 normalized_score', score_3_4/worst)

Обычная оценка / Стандартизированная оценка
1-3 score:  0.061069959441721305 1-3 normalized_score 0.2938594176016951
1-2 score:  0.10610140618899112 1-2 normalized_score 0.5105439354216643
1-4 score:  0.2061454031911393 1-4 normalized_score 0.9919405330672245
2-4 score:  0.20782032422216654 2-4 normalized_score 1.0
3-4 score:  0.1998273742277107 3-4 normalized_score 0.9615391323039651


Как можно заметить, 1-3 похожи, 1-2 чуть менее похожи, 1/2/3-4 - сильно отличаются.